In [ ]:


import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import time

import requests
from bs4 import BeautifulSoup

In [ ]:
def get_auto(suburl):
    """Function to parse particular unit"""
    sub_my_columns = ['bodyType', 'brand', 'color', 'fuelType',
                  'modelDate', 'name', 'numberOfDoors', 'productionDate',
                  'vehicleConfiguration', 'vehicleTransmission', 'engineDisplacement', 'enginePower',
                  'description', 'mileage', 'complectation', 'drive', 'hand', 'condition', 'owners',
                  'PTS', 'customs', 'owningTime', 'id' 'price']
    auto_parse = sub_my_columns[:13]
    
    car = pd.Series(index = sub_my_columns)
    sub_r = requests.get(suburl, headers={'User-Agent': 'Mozilla/5.0'})
    sub_r.encoding = 'utf-8'
    soup = BeautifulSoup(sub_r.text, 'html.parser')
    for chrs in auto_parse:
        #data we find is particularly located in meta with almost exactly the same itemprops as mentioned in test dataset
        if chrs == 'bodyType':
            try:
                car[chrs] = soup.find('meta', itemprop=chrs)['content'].split(' ')[0]
            except:
                car[chrs] = 'no_data'
        else:
            try:
                car[chrs] = soup.find('meta', itemprop=chrs)['content']
            except:
                car[chrs] = 'no_data'
                
            try:
                car['price'] = float(soup.find('meta', itemprop='price')['content'])
            except:
                car['price'] = 'no_data'
                
    #The rest of the data we'll awkwardly parse from tech list by html tags  
    try:
        car['mileage'] = float(soup.find('li', class_="CardInfo__row CardInfo__row_kmAge").text.replace(
            'Пробег','').replace('км','').replace(u'\xa0', u''))
    except:
        car['mileage'] = 0.0
   
    try:
        car['complectation'] = soup.find('li',
                                     class_="CardInfoGrouped__row CardInfoGrouped__row_complectation_name").text.replace(
                                    'Комплектация', '')
    except:
        car['complectation'] = 'no_data'
        
    try:
        car['hand'] = soup.find('li', class_="CardInfo__row CardInfo__row_wheel").text.replace(
            'Руль','')
    except:
        car['hand'] = 'левый'
    
    try:
        car['drive'] = soup.find('li', class_="CardInfo__row CardInfo__row_drive").text.replace(
            'Привод','')
    except:
        car['drive'] = 'no_data'
        
    try:
        car['condition'] = soup.find('li', class_="CardInfo__row CardInfo__row_state").text.replace(
            'Состояние','')
    except:
        car['condition'] = 'no_data'
        
    try:
        car['owners'] = soup.find('li', class_="CardInfo__row CardInfo__row_ownersCount").text.replace(
            'Владельцы','').replace(u'\xa0', u' ')
    except:
        car['owners'] = 'no_data'
        
    try:
        car['PTS'] = soup.find('li', class_="CardInfo__row CardInfo__row_pts").text.replace(
            'ПТС','')
    except:
        car['PTS'] = 'no_data'
    
    try:
        car['customs'] = soup.find('li', class_="CardInfo__row CardInfo__row_customs").text.replace(
            'Таможня','')
    except:
        car['customs'] = 'no_data'
    
    try:
        car['owningTime'] = soup.find('li', class_="CardInfo__row CardInfo__row_owningTime").text.replace(
            'Владение','')
    except:
        car['owningTime'] = 'no_data'
    
    try:
        car['id'] = int(soup.find('div', title="Идентификатор объявления").text.replace('№ ',''))
    except:
        car['id'] = 'no_data'
    return (car)
    
        
def parse_autoru(x):
    """General parsing function"""
    my_columns = ['bodyType', 'brand', 'color', 'fuelType',
                  'modelDate', 'name', 'numberOfDoors', 'productionDate',
                  'vehicleConfiguration', 'vehicleTransmission', 'engineDisplacement', 'enginePower',
                  'description', 'mileage', 'complectation', 'drive', 'hand', 'condition', 'owners',
                  'PTS', 'customs', 'owningTime', 'id' 'price']
    df = pd.DataFrame(columns = my_columns)
    url = 'https://auto.ru/moskva/cars/' + x + '/all/?page='
    suburl = []
    for i in range(1,100):
        r = requests.get(url+str(i), headers={'User-Agent': 'Mozilla/5.0'})
        r.encoding = 'utf-8'
        main_soup = BeautifulSoup(r.text, 'html.parser')
        links = main_soup.find_all('a', class_="Link ListingItemTitle-module__link")
        for link in links:
            df = df.append(get_auto(link['href']), ignore_index = True)
            df.to_excel('backup_train.xls')
        time.sleep(np.random.randint(1,2))
    return df
                   

In [ ]:
car_brand = 'mercedes'
data = parse_autoru(car_brand)
data.to_excel(car_brand + '_train.xls')

# Test Space